# Clone the Dataset Repository


In [22]:
!git clone https://huggingface.co/datasets/billsum

fatal: destination path 'billsum' already exists and is not an empty directory.


# Install Necessary Libraries

In [23]:
! pip install -q transformers[torch]

In [24]:
!pip install transformers datasets rouge_score

# Load the Dataset


In [25]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
import re
import string

# Load the dataset from parquet files
test_df = pd.read_parquet('/content/billsum/data/test-00000-of-00001.parquet')

# Function to normalize text
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Apply normalization
test_df['text'] = test_df['text'].apply(normalize_text)
test_df['summary'] = test_df['summary'].apply(normalize_text)

# Define Evaluation Metrics

In [27]:
!pip install evaluate

In [28]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [30]:
text = test_df.iloc[100]['text']
text = "summarize: " + text
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
inputs = tokenizer(text, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (1577 > 512). Running this sequence through the model will result in indexing errors


In [31]:
print("Original:", test_df.iloc[100]['summary'])
print("Generated Summary:", summary_text)

Original: afghanistan and central asian republics sustainable food production act of 2001 directs the administrator of the united states agency for international development to provide financial assistance to nongovernmental organizations carrying out rural developmental activities in afghanistan kyrgyzstan pakistan tajikistan turkmenistan and uzbekistan stipulates the aid shall be used for 1 restocking seed 2 replacing breeding livestock 3 restoring basic irrigation systems 4 providing access to credit for food production processing or marketing enterprises through rural microenterprise loan programs and 5 technical assistance places human rights and other conditions on the government of afghanistan for projects to be funded in afghanistan
Generated Summary: sec 2 findings congress finds 1 abject poverty and inability to produce food even at the subsistence level in the rural mountainous areas of afghanistan and the central asian republics. the government has been unable to produce fo

In [32]:
import evaluate

# Load the ROUGE metric
rouge = evaluate.load('rouge')

# Assume we have a reference summary (ground truth)
reference_summary = test_df.iloc[100]['summary']

# Compute the ROUGE score
scores = rouge.compute(predictions=[summary_text], references=[reference_summary])

print("ROUGE Scores:", scores)

ROUGE Scores: {'rouge1': 0.35838150289017345, 'rouge2': 0.10526315789473685, 'rougeL': 0.19653179190751446, 'rougeLsum': 0.19653179190751446}
